In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
from elasticsearch import Elasticsearch
from elasticsearch import helpers
indexingTables = Elasticsearch()

In [2]:
def get_accuracy(idRankedTables, idQueryGoal):
    
    accuracy = 0

    for idTable in idRankedTables:
    
        if idTable == idQueryGoal:
    
            accuracy = 1
            break;

    return accuracy

In [3]:
indexingTables.indices.close(index='tables')
indexingTables.indices.put_settings(index='tables', body={"index": {"similarity": {"default": {"type": "classic"}}}})
indexingTables.indices.open(index='tables')

{'acknowledged': True, 'shards_acknowledged': True}

In [5]:
def searchIndexing(query, tableField, topK):
    
    result= indexingTables.search(
        index="tables", 
        body = {
        "_source": ["tablePgID"],
        "from" : 0,
        "size" : topK,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    query, 
              "fields": ["tablePgTitle","tableHeader","tableBody"] 
              #"fields": [tableField]
            }
        }
    })
    
    return result

In [6]:
articles = pd.read_csv('dataset/test_dataset', delimiter=',', header=None)
formattedArticles = articles.iloc[:,:].values

In [13]:
def executeSearch(tableField, topK):

    accuracy = []

    for articles in (formattedArticles):
    
        articl_key = articles[0]
        article_title = str(articles[1])
        article_full_text = str(articles[2])
        article_meta_description = str(articles[3])
        article_summary = str(articles[4])
        keywords = str(articles[5])
        catch = article_title +" "+article_meta_description+" "+keywords
        
        result = searchIndexing(catch,tableField,topK)
    
        idRankedTables = []
    
        for hit in result['hits']['hits']:
    
            tablePgId = hit['_source']['tablePgID']
    
            idRankedTables.append(tablePgId)
    
        accuracy.append(getAccuracy(idRankedTables,articl_key))

    print(str(round(np.mean(accuracy),4)))

In [10]:
tableField = ["tablePgTitle","tableSectionTitle","tableCaption","tableHeader","tableBody"]
topK = [1,10,20,50,100,1000]

In [ ]:
for k in topK:
    
    print("")
    
    for field in tableField:
        
        print("Acc@"+str(k)+" - "+field)
        
        executeSearch(field,k)

In [ ]:
for k in topK:
    
    print("Acc@"+str(k))
    
    executeSearch("",k)